<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-25 11:47:01
-------------------
qualified stocks: 93
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.39 L
Current:  1.38 C
-------------------
Today PnL: 44.08 K (0.32%)
Current PnL: -17.79 L (-12.1%)
CY Booked + Current PnL: -9.06 L (-6.16%)
-------------------
Total profit:  3.67 L
Total loss:  -21.46 L
-------------------
Total Booked + Current PnL: 17.92 L (14.93%)
Total Booked PnL: 35.71 L (29.75%)
Curr Year Booked PnL: 8.73 L (6.35%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 76.90 L (55.91%)
Deployed:  1.20 C
Current:  1.38 C
CAGR/XIRR %: 8.62%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-17.42,H-LC,7.19,114448.0,10136.0,6512.0,1.58,9.72,5.69,15.96,9.0,1.56,0.85,28.26,X5K,ATH,METALS
4,APOLLOHOSP,7098.95,8285.00,-13.53,H-LC,7.87,172722.0,16545.0,9552.0,-0.90,10.59,5.53,16.71,22.0,1.73,1.29,29.88,X40N,BTT,HEALTHCARE
37,HINDUNILVR,2413.81,2723.00,-6.41,H-LC,7.74,294504.0,24157.0,10484.0,-0.02,8.94,3.56,12.81,24.0,2.30,2.20,23.17,XY25,NTT,FMCG
55,MEDANTA,1087.93,1486.00,13.28,H-MC,14.68,161727.0,35527.0,10642.0,0.69,28.15,6.58,36.59,108.0,3.34,1.21,44.24,XY24,NTT,HEALTHCARE
68,SBIN,760.30,863.00,-12.30,M-LC,3.96,216783.0,15304.0,11901.0,0.22,7.60,5.49,13.51,60.0,1.29,1.62,21.15,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,NATIONALUM,189.63,247.44,-49.46,H-MC,1.95,101416.0,912.0,29725.0,1.31,0.91,29.31,30.49,79.0,0.03,0.76,33.61,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-39.61,H-LC,3.19,145582.0,-4350.0,18940.0,0.41,-2.90,13.01,9.74,4.0,-0.23,1.09,3.16,X40,NTT,FMCG
53,LTIM,5564.16,7230.2,-3.47,H-LC,3.38,191430.0,-8880.0,68857.0,1.78,-4.43,35.97,29.94,16.0,-0.13,1.43,32.25,X200,ATH,IT
57,NESTLEIND,1134.30,1377.0,-11.53,H-LC,3.68,272119.0,6693.0,50097.0,-1.40,2.52,18.41,21.40,11.0,0.13,2.03,9.79,XY25,NTT,FMCG
21,CIPLA,1492.70,1795.0,-17.72,H-LC,4.18,219200.0,14700.0,26720.0,0.45,7.19,12.19,20.25,10.0,0.55,1.63,16.55,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-13.19,H-LC,6.07,169422.0,-31964.0,119578.0,-0.52,-15.87,70.58,43.51,27.0,-0.27,1.26,17.93,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,INFY,1461.46,2275.00,-16.01,H-LC,11.98,274783.0,13182.0,132445.0,3.20,5.04,48.20,55.67,3.0,0.10,2.05,11.45,X40,BTT,IT
80,TCS,3794.03,4998.00,-25.80,H-LC,16.10,245326.0,-50608.0,144522.0,2.99,-17.10,58.91,31.73,1.0,-0.35,1.83,4.74,X40,BTT,IT
92,WIPRO,243.46,420.00,-10.16,M-LC,7.77,158261.0,7316.0,102142.0,2.66,4.85,64.54,72.51,53.0,0.07,1.18,11.16,XR,NTT,IT
34,HCLTECH,1562.67,1943.91,3.23,H-LC,10.56,183403.0,-7243.0,53755.0,2.52,-3.80,29.31,24.40,8.0,-0.13,1.37,11.53,X40,ATH,IT
17,BSOFT,628.85,836.99,-12.86,H-SC,16.22,67331.0,-42718.0,79141.0,2.49,-38.82,117.54,33.10,131.0,-0.54,0.50,11.88,XR,ATH,IT


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.0,-43.66,L-SC,25.81,83501.0,-30048.0,70099.0,-1.89,-26.46,83.95,35.27,268.0,-0.43,0.62,104.37,XR,NTT,HOTELS
75,SURYODAY,189.78,240.0,42.54,H-SC,5.41,101878.0,-48048.0,87717.0,-1.69,-32.05,86.10,26.46,135.0,-0.55,0.76,30.51,XR,NTT,BANKS
6,ASIANTILES,75.41,137.0,6816.67,L-SC,6.81,75896.0,-17914.0,94528.0,-1.68,-19.10,124.55,81.67,269.0,-0.19,0.57,46.98,XR,NTT,CERAMICS
70,SHALBY,261.39,327.0,895.22,M-SC,5.50,135412.0,-45731.0,91200.0,-1.47,-25.25,67.35,25.10,235.0,-0.50,1.01,8.52,XY24,NTT,HEALTHCARE
57,NESTLEIND,1134.30,1377.0,-11.53,H-LC,3.68,272119.0,6693.0,50097.0,-1.40,2.52,18.41,21.40,11.0,0.13,2.03,9.79,XY25,NTT,FMCG


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,100.21,M-SC,8.85,86270.0,-14507.0,14597.0,0.61,-14.40,16.92,0.09,245.0,-0.99,0.64,13.34,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,87.86,H-SC,15.74,119016.0,-23463.0,42989.0,0.85,-16.47,36.12,13.70,163.0,-0.55,0.89,40.47,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-18.38,H-SC,3.09,116334.0,-27469.0,54212.0,-1.00,-19.10,46.60,18.60,149.0,-0.51,0.87,28.38,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,113.73,M-SC,1.81,140132.0,-34427.0,34458.0,-0.30,-19.72,24.59,0.02,221.0,-1.00,1.04,21.89,OX40N,NTT,PAINTS
50,KANSAINER,299.63,340.0,-69.62,H-SC,3.55,209817.0,-59850.0,96180.0,0.22,-22.19,45.84,13.47,138.0,-0.62,1.56,6.12,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-25.80,H-LC,16.10,245326.0,-50608.0,144522.0,2.99,-17.10,58.91,31.73,1.0,-0.35,1.83,4.74,X40,BTT,IT
45,INFY,1461.46,2275.00,-16.01,H-LC,11.98,274783.0,13182.0,132445.0,3.20,5.04,48.20,55.67,3.0,0.10,2.05,11.45,X40,BTT,IT
47,ITC,411.90,452.00,-39.61,H-LC,3.19,145582.0,-4350.0,18940.0,0.41,-2.90,13.01,9.74,4.0,-0.23,1.09,3.16,X40,NTT,FMCG
89,VBL,492.64,671.64,-8.30,H-LC,8.95,278603.0,13563.0,82745.0,0.24,5.12,29.70,36.33,5.0,0.16,2.08,19.10,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.42,H-LC,12.56,198842.0,-17677.0,110576.0,0.70,-8.16,55.61,42.91,7.0,-0.16,1.48,4.13,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,13.28,H-MC,14.68,161727.0,35527.0,10642.0,0.69,28.15,6.58,36.59,108.0,3.34,1.21,44.24,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-7.35,H-LC,10.29,231373.0,46892.0,23484.0,0.49,25.42,10.15,38.14,38.0,2.00,1.72,38.72,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,13.28,H-MC,14.68,161727.0,35527.0,10642.0,0.69,28.15,6.58,36.59,108.0,3.34,1.21,44.24,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,11.32,H-MC,5.01,192120.0,27450.0,40518.0,0.06,16.67,21.09,41.28,89.0,0.68,1.43,25.98,X40N,ATH,AC
51,KPIGREEN,497.21,731.35,8.64,H-SC,10.90,129452.0,4155.0,54849.0,-0.99,3.32,42.37,47.09,141.0,0.08,0.97,62.02,MH,ATH,POWER
90,VOLTAS,1278.28,1918.49,-2.34,H-MC,6.95,204900.0,13158.0,82882.0,0.95,6.86,40.45,50.08,99.0,0.16,1.53,14.13,XY25,ATH,AC
19,CAMS,3643.00,5211.76,-4.72,H-SC,5.83,108402.0,6398.0,37529.0,-0.86,6.27,34.62,43.06,122.0,0.17,0.81,24.46,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-4.90,H-MC,13.11,176368.0,25455.0,31852.0,0.82,16.87,18.06,37.97,93.0,0.80,1.31,46.44,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-49.60,H-SC,12.30,140503.0,17167.0,116589.0,-0.73,13.92,82.98,108.45,119.0,0.15,1.05,39.38,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-42.05,M-SC,5.50,97965.0,6466.0,72288.0,0.40,7.07,73.79,86.07,223.0,0.09,0.73,41.72,XR,NTT,DURABLES
51,KPIGREEN,497.21,731.35,8.64,H-SC,10.90,129452.0,4155.0,54849.0,-0.99,3.32,42.37,47.09,141.0,0.08,0.97,62.02,MH,ATH,POWER
92,WIPRO,243.46,420.00,-10.16,M-LC,7.77,158261.0,7316.0,102142.0,2.66,4.85,64.54,72.51,53.0,0.07,1.18,11.16,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,75.41,137.00,6816.67,L-SC,6.81,75896.0,-17914.0,94528.0,-1.68,-19.10,124.55,81.67,269.0,-0.19,0.57,46.98,XR,NTT,CERAMICS
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,25.81,83501.0,-30048.0,70099.0,-1.89,-26.46,83.95,35.27,268.0,-0.43,0.62,104.37,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,0.54,H-SC,16.10,93564.0,-7399.0,101929.0,1.17,-7.33,108.94,93.63,148.0,-0.07,0.70,39.36,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-15.54,H-SC,12.72,96045.0,-1935.0,23810.0,1.99,-1.97,24.79,22.32,152.0,-0.08,0.72,39.30,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-17.42,H-LC,7.19,114448.0,10136.0,6512.0,1.58,9.72,5.69,15.96,9.0,1.56,0.85,28.26,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,25.81,83501.0,-30048.0,70099.0,-1.89,-26.46,83.95,35.27,268.0,-0.43,0.62,104.37,XR,NTT,HOTELS
35,HEROMOTOCO,4311.81,5949.07,-4.90,H-MC,13.11,176368.0,25455.0,31852.0,0.82,16.87,18.06,37.97,93.0,0.80,1.31,46.44,AR,ATH,AUTO
28,DMART,3764.92,5201.00,-7.35,H-LC,10.29,231373.0,46892.0,23484.0,0.49,25.42,10.15,38.14,38.0,2.00,1.72,38.72,X40N,NTT,FMCG
15,BLUESTARCO,1646.70,2326.38,11.32,H-MC,5.01,192120.0,27450.0,40518.0,0.06,16.67,21.09,41.28,89.0,0.68,1.43,25.98,X40N,ATH,AC
37,HINDUNILVR,2413.81,2723.00,-6.41,H-LC,7.74,294504.0,24157.0,10484.0,-0.02,8.94,3.56,12.81,24.0,2.30,2.20,23.17,XY25,NTT,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.88
1,25,41.47
2,50,71.11


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.45
LC    36.33
MC    26.29
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.34
X40      14.41
X40N     14.00
XY25     11.81
XR        9.06
AR        8.47
OX40N     7.43
X5K       2.26
MH        1.73
X200      1.43
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.08
H-MC    23.39
H-SC    22.70
M-SC    13.20
M-LC     7.20
M-MC     2.57
L-SC     1.55
L-LC     1.05
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.52,0.13,30.45
IT,12.11,-15.01,72.45
FINANCE,8.06,-20.70,65.36
BANKS,7.54,-12.87,60.80
MISC,5.89,-16.50,83.28
PAINTS,5.45,-19.05,42.51
ELECTRICAL,4.76,-10.03,57.69
AUTO,4.54,-11.10,59.44
AC,3.69,2.28,30.82


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2835857.0,23
XR,1077922.0,13
AR,1075363.0,10
X40,805007.0,11
X40N,490008.0,11
XY25,464186.0,8
OX40N,461722.0,10
SR,192030.0,2
X5K,134731.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2735506.0,27
M-SC,1474570.0,17
H-MC,1242166.0,18
H-LC,1134727.0,19
M-LC,413678.0,5
M-MC,333832.0,2
L-SC,253146.0,3
L-MC,58848.0,1
L-LC,43784.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      930993.0      6
M-SC       XY24      866297.0      7
H-SC       AR        654741.0      5
           XR        517167.0      6
H-LC       X40       508468.0      6
H-MC       XY24      485674.0      5
M-MC       XY24      333832.0      2
H-LC       AR        229780.0      2
H-MC       X40       213787.0      4
H-SC       OX40N     199127.0      4
H-MC       XY25      198475.0      2
H-SC       SR        192030.0      2
           X40N      186599.0      3
H-LC       X40N      185568.0      5
M-LC       XY24      171416.0      2
L-SC       XR        164627.0      2
M-SC       AR        158990.0      2
M-LC       X5K       128219.0      1
M-SC       OX40N     123874.0      4
           XY25      122129.0      1
           XR        120528.0      2
H-MC       X40N      117841.0      3
           XR        114610.0      1
M-LC       XR        102142.0      1
L-SC       OX40N      88519.0      1
H-LC       XY25       87897.0      3
M-SC       X40        82752.0      1
H-LC       X200       68857.0      1
L-MC       XR         58848.0      1
H-SC       MH         54849.0      1
H-MC       OX40N      50202.0      1
H-LC       XY24       47645.0      1
L-LC       XY25       43784.0      1
H-MC       AR         31852.0      1
           MH         29725.0      1
M-LC       XY25       11901.0      1
H-LC       X5K         6512.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
